# *Detect Mask From Webcam*

In [1]:
# Import the necessary packages
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [3]:
# load face detector model
model = load_model('model.h5')
# Load the cascade cascadeClassifier
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# To capture video from webcam. 
source = cv2.VideoCapture(0)

# create label dict
labels_dict={1:'MASK',0:'NO MASK'}
color_dict={1:(0,255,0),0:(0,0,255)}

while(True):
    # Read the frame
    check, img = source.read()
    img = cv2.flip(img, 1)
    # convert to gray scale image
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
     # search the co-ordintes of the image(this is help to figure out where is the face in the image)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        # resize the frame
        resized=cv2.resize(face_img,(100,100))
        # normalized the frame
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        
        # method to create face rectangle into the image
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    # Display    
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    # if user press q exit the camera window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object
source.release()
cv2.destroyAllWindows()